In [132]:
import regex as re

In [133]:
input_file = "input.py"

with open(input_file) as f:
    text = f.read()

print(text)

from highlighting_functions import Highlighter
import json as jorre

this_variable = "hej på dig"
trippeljävel = """wtf"""

a_longer123string = this_variable + 'ööhö'


In [134]:
def tokenize(text) -> list[str]:
    """Tokenize a text"""
    # Possible string delimiters
    string_delims = {'"', "'", '"""', "'''"}

    # split everything
    tokens = re.findall(r"(\w+|[^\w\s]+|\s+|\n)", text)

    # reunite string literals
    tokens_merged = []
    current_delim = ""
    current_string = ""
    strs_count = 0
    for t in tokens:
        if current_delim == "":
            if t in string_delims:
                # start a string
                strs_count += 1
                current_delim = t
                current_string += t
            else:
                tokens_merged.append(t)
        else:
            if t == current_delim:
                # break current string
                current_delim = ""
                current_string += t
                tokens_merged.append(current_string)
                current_string = ""
            else:
                # add to current string
                current_string += t

    print(tokens)
    return tokens_merged


tokens = tokenize(text)
print(tokens)

['from', ' ', 'highlighting_functions', ' ', 'import', ' ', 'Highlighter', '\n', 'import', ' ', 'json', ' ', 'as', ' ', 'jorre', '\n\n', 'this_variable', ' ', '=', ' ', '"', 'hej', ' ', 'på', ' ', 'dig', '"', '\n', 'trippeljävel', ' ', '=', ' ', '"""', 'wtf', '"""', '\n\n', 'a_longer123string', ' ', '=', ' ', 'this_variable', ' ', '+', ' ', "'", 'ööhö', "'"]
['from', ' ', 'highlighting_functions', ' ', 'import', ' ', 'Highlighter', '\n', 'import', ' ', 'json', ' ', 'as', ' ', 'jorre', '\n\n', 'this_variable', ' ', '=', ' ', '"hej på dig"', '\n', 'trippeljävel', ' ', '=', ' ', '"""', 'wtf', '"""', '\n\n', 'a_longer123string', ' ', '=', ' ', 'this_variable', ' ', '+', ' ', "'ööhö'"]


In [136]:
classes = ("var", "op", "num")
